## Import Libraries

In [1]:
from time import sleep

In [9]:
%matplotlib ipympl
# %matplotlib inline

In [10]:
import matplotlib.pyplot as plt

In [11]:
from pydgilib_extra import *

Set the path to `dgilib.dll`.

In [12]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

## Data Logging

Create a figure for the plot.

In [13]:
fig = plt.figure(figsize=(10, 6))

<Figure size 720x432 with 0 Axes>

Create the configuration dictionary for `DGILibExtra`.

In [14]:
config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "augment_gpio" : True,
    "fig": fig,
    "verbose": 0,
}

Perform the measurement.

In [15]:
data = []
fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger_start()
    sleep(1)
    dgilib.update_callback()
    while not all(dgilib.data[INTERFACE_GPIO][1][-1]):
        dgilib.update_callback()
#         sleep(1)
    sleep(0.1)
    dgilib.logger_stop()
    data = dgilib.data

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\figure.py:445: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


## Analysis

In [9]:
# def power_and_time_per_pulse(data, pin, verbose=0, power_factor=1e3):
#     """Calculate power and time per pulse.
    
#     Takes the data and a pin and returns a list of power and time sums for each pulse of the specified pin.
#     It stops when all pins are high.
    
#     :param data: Data structure holding the samples.
#     :type data: dict(256:list(list(float), list(float)), 48:list(list(float), list(list(bool))))
#     :param pin: Number of the pin to be used.
#     :type pin: int
#     :return: List of list of power and time sums.
#     :rtype: tuple(list(float), list(float))
#     """
#     pin_value = False

#     charges = []
#     times = []

#     power_index = 0
#     power_sum = 0
#     time_sum = 0

#     start_time = 0
#     end_time = 0
#     last_time = 0

#     for timestamp, pin_values in zip(*data[INTERFACE_GPIO]):
#         if all(pin_values):
#             if verbose:
#                 print(f"power_and_time_per_pulse done, charges: {len(currents)}, times: {len(times)}")
#             break
#         if not pin_value and pin_values[pin]:
#             pin_value = True
#             start_time = timestamp
#             last_time = timestamp
#         if pin_value and not pin_values[pin]:
#             pin_value = False
#             end_time = timestamp
#             while (power_index < len(data[INTERFACE_POWER][0]) and data[INTERFACE_POWER][0][power_index] <= end_time):
#                 if (data[INTERFACE_POWER][0][power_index] >= start_time):
#                     power_sum += data[INTERFACE_POWER][1][power_index] * (data[INTERFACE_POWER][0][power_index] - last_time)
#                     time_sum += (data[INTERFACE_POWER][0][power_index] - last_time)
#                 last_time = data[INTERFACE_POWER][0][power_index]
#                 power_index += 1

#             charges.append(power_sum*power_factor)
#             times.append(time_sum)
#             power_sum = 0
#             time_sum = 0
                
#     return charges, times

In [16]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2)

In [17]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3)

In [18]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 0
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)

In [19]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [20]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 64
    # variables        = 2
    chi-square         = 2.6473e-08
    reduced chi-square = 4.2698e-10
    Akaike info crit   = -1378.78590
    Bayesian info crit = -1374.46813
[[Variables]]
    slope:      3.3331e-05 +/- 8.7389e-09 (0.03%) (init = 0)
    intercept:  4.0173e-04 +/- 5.1059e-06 (1.27%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.863

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 64
    # variables        = 2
    chi-square         = 1.9586e-08
    reduced chi-square = 3.1590e-10
    Akaike info crit   = -1398.07024
    Bayesian info crit = -1393.75248
[[Variables]]
    slope:      3.2335e-05 +/- 7.5167e-09 (0.02%) (init = 0)
    intercept:  3.8560e-04 +/- 4.3918e-06 (1.14%) (init = 1)
[[Correlations]] (unreporte

In [26]:
fig = plt.figure(figsize=(9, 6))

<Figure size 648x432 with 0 Axes>

In [27]:
charge_color = 'r'
time_color = 'b'

In [28]:
fig.clf()
# fig.suptitle("Energy analysis of AES")
ax1 = fig.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [29]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
fig.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig.tight_layout()
fig.show()

In [30]:
# data